# The Effects of Policy H6 on Social Housing

Importing required packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Affordable Housing in Coventry

We begin our analysis looking at the delivery of different components of affordable housing (affordable rent and social rent) within Coventry, comparing them to the overall housing market and the housing requirements set out by the Local Plan. 


Primarily, we are interested in how Policy H6 affects social housing in Coventry specifically, since there are no explicit requirements on social housing within the plan. The Local Plan sets out that new developments with >25 dwellings or >1ha in area must provide 25% of its dwellings as affordable housing. Further, the Local Plan assesses affordable housing need at the first 4 digits of the postcode level to determine areas of high existing stock of social housing (>1,500 properties) and areas of low existing stock of social housing (<500 properties). Anything in between is determined as medium concentration. Within the geographic areas, the provision of affordable housing is as follows:

* In areas of high concentration: 10% Social/Affordable Rental Provision; 15% Intermediate Provision
* In areas of medium concentration: 12.5% Social/Affordable Rental Provision; 12.5% Intermediate Provision
* In areas of low concentration: 15% Social/Affordable Rental Provision; 10% Intermediate Provision

We assume the intention of this policy is to bring up the level of affordable housing in areas of low concentration but is this setting up misaligned incentives? Areas of high concentration of social housing require fewer social houses making it more attractive to developers (because it gets higher rents) reducing the concentration of social housing and areas of low concentration don’t get as much development (because of the higher social/affordable housing requirement).

The Local Plan also recommends that the housing mix of affordable housing should follow the latest SHMA/SHLAA which suggests that houses with 3 or more bedrooms should make up 50% of the housing mix. However, as the 2021 Homefinder data suggests, houses with 3 or more bedrooms only make 14% of socially rented homes. We assume that since Policy H6 does not set out explicit requirements on the size of homes provided within the plan, developers might be building larger homes for the more profitable Intermediate Homes or Affordable Rent sector rather than the social rent sectors which is mostly made up of bedsits, 1BR, and 2BR homes. 

Finally the Council understands that it may not always be possible for developers to provide the full 25% at every site and thus allows developers to make viability arguments and reach alternative solutions such as off-site delivery or commuted sum payments through the S106 developer's contribution agreement. We expect that this avenue has a significant effect on the delivery of socially rented homes in Coventry and may be contributing to the undersupply problem of homes for Coventry's social housing market.



In this report we hope to answer the following questions:

* How has Policy H6 affected social housing specifically?
* Has Policy H6 succesfully rebalanced the concetration of Affordable Housing (and specifically social housing) in Coventry?
* How has Policy H6 affected the housing mix of affordable housing and social housing?
* How have viability arguments/settlements affected the delivery of social housing in Coventry?



## Exploring Housing Supply in Coventry

We begin this analysis by exploring the housing market of Coventry. We compare net dwellings per year to additional affordable, social, and intermediate housing. We try to identify any trends so it may inform analysis going forward. We also compare affordable housing delivery to the target set by the Council to meet their housing need of 348 homes/year. We also look at how social housing and affordable housing diverges in their delivery and the falling delivery of social housing. Finally, we compare the affordable housing market in Coventry with the rest of the UK.

### Data

For our data on net dwellings, we use the Department for Levelling Up, Housing, and Communities (DLUHC) and Ministry of Housing, Communities and Local Government (MHCLG) Live Table 122 available [here](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1035591/Live_Table_122.ods 'Live Table 122'). For our data on additional dwellings for social housing, affordable housing, and intermediate rent - we use DLUHC Live Tables 1006-1008 available [here](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1034087/Live_Tables_1006-1009.ods 'Live Table 1006-1009').

The data is prepared for analysis prior to importing, the process is documented in the files titled `netdwellings.ipynb`, `socialrent.ipynb`, `totalaffordablerent.ipynb`, `affordablerent.ipynb`, and `intermediaterent.ipynb`. 

#### Importing data

In [3]:
df1 = pd.read_excel('Net Dwellings by LA, 2001-2021.xlsx')
df2 = pd.read_excel('Socially Rented Dwellings by LA, 1992-2021.xlsx')
df3 = pd.read_excel('Total Affordable Rent Dwellings by LA, 1992-2021.xlsx')

In [7]:
for x in [df1,df2,df3]:
    x.drop(columns='Unnamed: 0',inplace=True)

In [8]:
df1.head()

,LA_code,LA,year,net dwellings,date
0,E06000022,Bath and North East Somerset UA,2002,270.4,2002-01-01
1,E06000055,Bedford UA,2002,NaN,2002-01-01
2,E06000008,Blackburn with Darwen UA,2002,-73.8,2002-01-01
3,E06000009,Blackpool UA,2002,159.7,2002-01-01
4,E06000028,Bournemouth UA,2002,764.3,2002-01-01


In [9]:
df2.head()

,LA_code,LA,year,new_dwellings
0,E07000157,Alnwick,1992,44.0
1,E07000158,Berwick-upon-Tweed,1992,5.0
2,E07000159,Blyth Valley,1992,16.0
3,E07000160,Castle Morpeth,1992,8.0
4,E07000054,Chester-le-Street,1992,0.0


In [10]:
df3.head()

,LA_code,LA,year,new_dwellings
0,E07000157,Alnwick,1992,45.0
1,E07000158,Berwick-upon-Tweed,1992,5.0
2,E07000159,Blyth Valley,1992,21.0
3,E07000160,Castle Morpeth,1992,8.0
4,E07000054,Chester-le-Street,1992,16.0
